In [104]:
""" Query wsdb for Gaia and GALEX magnitudes. """

##### Dependencies
import numpy as np
import pandas as pd
%matplotlib osx
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord as coord
import sqlutilpy
from astropy.io import fits

In [ ]:
##### Load your wsdb.id
wsdbtmp = np.loadtxt('./wsdb.id',dtype=np.str_)
wsdbid = {'user':wsdbtmp[0],'password':wsdbtmp[1]}

In [106]:
##### Acquire QSOs from table
fullqso_keys = ['ra','dec','galex_matched','nuv','fuv','psfmag_u', 'psfmag_g', 'psfmag_r', 'psfmag_i', 'psfmag_z', 'err_psfmag_u', 'err_psfmag_g', 'err_psfmag_r', 'err_psfmag_i', 'err_psfmag_z', 'extinction_recal_u', 'extinction_recal_g', 'extinction_recal_r', 'extinction_recal_i', 'extinction_recal_z']
fullqso_x = sqlutilpy.get('select ra, dec, galex_matched, nuv, fuv, psfmag_u, psfmag_g, psfmag_r, psfmag_i, psfmag_z, err_psfmag_u, err_psfmag_g, err_psfmag_r, err_psfmag_i, err_psfmag_z, extinction_recal_u, extinction_recal_g, extinction_recal_r, extinction_recal_i, extinction_recal_z from sdssdr12qso.main limit 300000 ',
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='douglas_boubert', password='chopel3082')

In [122]:
##### Build colours
fullqsobox = {k:v for k,v in zip(fullqso_keys,fullqso_x)}
fullqsobox['gr0'] = (fullqsobox['psfmag_g']-fullqsobox['extinction_recal_g'])-(fullqsobox['psfmag_r']-fullqsobox['extinction_recal_r'])
fullqsobox['ug0'] = (fullqsobox['psfmag_u']-fullqsobox['extinction_recal_u'])-(fullqsobox['psfmag_g']-fullqsobox['extinction_recal_g'])
fullqso_x = [v for k,v in fullqsobox.items()]
fullqso_keys = [k for k,v in fullqsobox.items()]

In [123]:
##### Select a random sample
goodphot = np.where(fullqsobox['gr0']>-100)
nchoose = 15000
qsochoice = np.random.choice(range(goodphot[0].shape[0]),nchoose, replace=False)
qso_x = [fullqso_x[i][goodphot[0][qsochoice]] for i in range(len(fullqso_keys))]
qso_keys = ['qso'+fullqso_keys[i] for i in range(len(fullqso_keys))]
qsobox = {k:v for k,v in zip(qso_keys,qso_x)}

In [124]:
gaia_keys = ['source_id', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', 'phot_g_mean_flux_error', 'phot_g_mean_flux_over_error', 'phot_g_mean_mag', 'phot_bp_n_obs', 'phot_bp_mean_flux', 'phot_bp_mean_flux_error', 'phot_bp_mean_flux_over_error', 'phot_bp_mean_mag', 'phot_rp_n_obs', 'phot_rp_mean_flux', 'phot_rp_mean_flux_error', 'phot_rp_mean_flux_over_error', 'phot_rp_mean_mag', 'phot_bp_rp_excess_factor', 'phot_proc_mode', 'bp_rp', 'bp_g', 'g_rp', 'phot_variable_flag', 'l', 'b', 'a_g_val', 'e_bp_min_rp_val', 'radial_velocity', 'radial_velocity_error', 'rv_nb_transits','rv_template_teff','rv_template_logg','rv_template_fe_h','ebv']
galex_keys = ['galex_ra','galex_dec','fuv_mag','fuv_magerr','nuv_mag','nuv_magerr']
allwise_keys = ['allwise_ra','allwise_dec','w1mpro','w1sigmpro','w2mpro','w2sigmpro']

In [125]:
### Cross-match with Gaia source
gaia_x = sqlutilpy.local_join("""
   select CAST(source_id as real), ra, ra_error, dec, dec_error, parallax, parallax_error, parallax_over_error, pmra, pmra_error, pmdec, pmdec_error, ra_dec_corr, ra_parallax_corr, ra_pmra_corr, ra_pmdec_corr, dec_parallax_corr, dec_pmra_corr, dec_pmdec_corr, parallax_pmra_corr, parallax_pmdec_corr, pmra_pmdec_corr, CAST(astrometric_n_obs_al as real), CAST(astrometric_n_obs_ac as real), CAST(astrometric_n_good_obs_al as real), CAST(astrometric_n_bad_obs_al as real), astrometric_gof_al, astrometric_chi2_al, astrometric_excess_noise, astrometric_excess_noise_sig, CAST(astrometric_params_solved as real), astrometric_primary_flag, astrometric_weight_al, astrometric_pseudo_colour, astrometric_pseudo_colour_error, mean_varpi_factor_al, CAST(astrometric_matched_observations as real), CAST(visibility_periods_used as real), astrometric_sigma5d_max, CAST(frame_rotator_object_type as real), CAST(matched_observations as real), duplicated_source, CAST(phot_g_n_obs as real), phot_g_mean_flux, phot_g_mean_flux_error, phot_g_mean_flux_over_error, phot_g_mean_mag, CAST(phot_bp_n_obs as real), phot_bp_mean_flux, phot_bp_mean_flux_error, phot_bp_mean_flux_over_error, phot_bp_mean_mag, CAST(phot_rp_n_obs as real), phot_rp_mean_flux, phot_rp_mean_flux_error, phot_rp_mean_flux_over_error, phot_rp_mean_mag, phot_bp_rp_excess_factor, CAST(phot_proc_mode as real), bp_rp, bp_g, g_rp, phot_variable_flag, l, b, a_g_val, e_bp_min_rp_val, radial_velocity, radial_velocity_error, CAST(rv_nb_transits as real),rv_template_teff,rv_template_logg,rv_template_fe_h,ebv from mytable as m left join lateral (select * from gaia_dr2.gaia_source as g where 
               q3c_join(m.qsora, m.qsodec,g.ra,g.dec,1./3600)
       order by q3c_dist(m.qsora,m.qsodec,g.ra,g.dec) asc limit 1)
       as tt on  true  order by xid""",'mytable',
       (qsobox['qsora'],qsobox['qsodec'],np.arange(len(qsobox['qsora']))),('qsora','qsodec','xid'),
           host='cappc127.ast.cam.ac.uk', user=wsdbid['user'], password=wsdbid['password'],
       preamb='set enable_seqscan to off; set enable_mergejoin to off; set enable_hashjoin to off;')
print("Finished Gaia cross-match, "+str(np.where(np.isnan(gaia_x[0])==False)[0].shape[0])+" sources.")                

### Cross-match with GALEX
galex_x = sqlutilpy.local_join("""
   select ra,dec,fuv_mag,fuv_magerr,nuv_mag,nuv_magerr from mytable as m left join lateral (select * from galex_bst17.main_fov055 as g where 
               q3c_join(m.qsora, m.qsodec,g.ra,g.dec,10./3600)
       order by q3c_dist(m.qsora,m.qsodec,g.ra,g.dec) asc limit 1)
       as tt on  true  order by xid""",'mytable',
       (qsobox['qsora'],qsobox['qsodec'],np.arange(len(qsobox['qsora']))),('qsora','qsodec','xid'),
           host='cappc127.ast.cam.ac.uk', user=wsdbid['user'], password=wsdbid['password'],
       preamb='set enable_seqscan to off; set enable_mergejoin to off; set enable_hashjoin to off;')
print("Finished GALEX cross-match, "+str(np.where(np.isnan(galex_x[0])==False)[0].shape[0])+" sources.")

### Cross-match with ALLWISE
allwise_x = sqlutilpy.local_join("""
   select ra, dec, w1mpro, w1sigmpro, w2mpro, w2sigmpro from mytable as m left join lateral (select * from allwise.main as g where 
               q3c_join(m.qsora, m.qsodec,g.ra,g.dec,10./3600)
       order by q3c_dist(m.qsora,m.qsodec,g.ra,g.dec) asc limit 1)
       as tt on  true  order by xid""",'mytable',
       (qsobox['qsora'],qsobox['qsodec'],np.arange(len(qsobox['qsora']))),('qsora','qsodec','xid'),
           host='cappc127.ast.cam.ac.uk', user=wsdbid['user'], password=wsdbid['password'],
       preamb='set enable_seqscan to off; set enable_mergejoin to off; set enable_hashjoin to off;')
print("Finished ALLWISE cross-match, "+str(np.where(np.isnan(allwise_x[0])==False)[0].shape[0])+" sources.") 

Finished Gaia cross-match, 9695 sources.
Finished GALEX cross-match, 4128 sources.
Finished ALLWISE cross-match, 12723 sources.


In [128]:
##### Merge together
x = qso_x+gaia_x+galex_x+allwise_x
keys = qso_keys+gaia_keys+galex_keys+allwise_keys
gagaqsobox = {keys[i]:x[i] for i in range(len(x))}

In [129]:
##### Save cross-match
np.savez_compressed('./data/gagaqso.npz',gagaqsobox=gagaqsobox)